In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import qgrid

imdbExport = (r'C:\Users\User\Downloads\ratings.csv') # Change to your imdb export file directory 
imdb_df = pd.read_csv(imdbExport,encoding = "ISO-8859-1",na_values='0', engine='python', parse_dates=False)

filterBy = input("Please choose by typing one of following rating item type to analyze: tvEpisode,movie,tvSpecial,tvSeries,short,tvMiniSeries,tvMovie,video")

imdb_dfFiltered = imdb_df.loc[imdb_df['Title Type']== filterBy]

dateRatedObjFiltered = imdb_dfFiltered['Date Rated'].astype('datetime64[ns]') #to convert dtype to date from obj
yearRatedFiltered = dateRatedObjFiltered.dt.strftime("%Y") #to remove day of month from date rated column

'''Yearly data prep'''

ratingsByYrPresortFiltered = pd.value_counts(yearRatedFiltered.values, sort=False) # to create a tally by mth year
ratingsByYrFiltered = ratingsByYrPresortFiltered.sort_index() # to sort by month year which is the index

sumOfYourRatingsByYearFiltered = imdb_dfFiltered['Your Rating'].groupby(yearRatedFiltered).sum() 
yourAvgRatingByYrFiltered = (sumOfYourRatingsByYearFiltered/ratingsByYrFiltered).round(decimals=2) #sum to divide by count total

'''Monthly data prep'''

dateRatedObj = imdb_dfFiltered['Date Rated'].astype('datetime64[ns]') #to convert dtype to date from obj
yearMthRated = dateRatedObj.dt.strftime("%Y/%m") #to remove day of month from date rated column

ratingsByDatePresort = pd.value_counts(yearMthRated.values, sort=False) # to create a tally by mth year
ratingsByDate = ratingsByDatePresort.sort_index() # to sort by month year which is the index

sumOfYourRatings = imdb_dfFiltered['Your Rating'].groupby(yearMthRated).sum() 

yourAvgRatingByMth = (sumOfYourRatings/ratingsByDate).round(decimals=2) #sum to divide by count total

'''Director table data prep'''

dirCount = pd.value_counts(imdb_dfFiltered['Directors'].values)
dirCountDF = dirCount.to_frame() 
dirCountDF.columns = ['Count'] # add header to this df 
dirCountDF.index.name = 'Directors'

sumOfYourDirRatings = imdb_dfFiltered['Your Rating'].groupby(imdb_dfFiltered['Directors']).sum()
sumOfYourDirRatings.to_frame()

dirRating = (sumOfYourDirRatings/dirCount).round(decimals=2)
dirRating2 = dirRating.to_frame()
dirRating2.columns = ['Avg Rating']

dirDF = pd.merge(dirRating2,dirCountDF, on='Directors')
dirDF.sort_values("Count", inplace=True, ascending=False)
dfDirDF = dirDF.head(n=25)

'''Yearly graph prep'''

fontStyle = dict(family="Arial Narrow", size=11,color="#7f7f7f")
legendStyle = dict(x=0.73,y=0.99) 

# Create figure with secondary y-axis
fig2Filtered = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig2Filtered.add_trace(
    go.Bar(x=ratingsByYrFiltered.index, y=ratingsByYrFiltered, name="Items Rated Filtered",marker_color='#02075d'),
    secondary_y=False,
)

fig2Filtered.add_trace(
    go.Scatter(x=yourAvgRatingByYrFiltered.index, y=yourAvgRatingByYrFiltered, name="Your Avg Rating",),
    secondary_y=True,
)

# figure title
fig2Filtered.update_layout(
    title_text="Total Items Rated vs Avg Rating by Year",
    font=fontStyle,
    legend=legendStyle
)

# Set x-axis title
fig2Filtered.update_xaxes(title_text="Year")

# Set y-axes titles
fig2Filtered.update_yaxes(title_text="Count of Items Rated", secondary_y=False)
fig2Filtered.update_yaxes(title_text="Your Avg Rating", secondary_y=True)

'''Monthly graph prep'''

# Monthly Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Monthly Add traces
fig.add_trace(
    go.Bar(x=ratingsByDate.index, y=ratingsByDate, name="Items Rated",marker_color='#02075d'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=yourAvgRatingByMth.index, y=yourAvgRatingByMth, name="Your Avg Rating"),
    secondary_y=True,   
)

# Monthly figure title  
    
fig.update_layout(
    title_text="Total Items Rated vs Avg Rating by Month",
    font=fontStyle,
    legend=legendStyle
)

# Monthly Set x-axis title
fig.update_xaxes(title_text="Month")

# Monthly Set y-axes titles
fig.update_yaxes(title_text="Items Rated", secondary_y=False)
fig.update_yaxes(title_text="Your Avg Rating", secondary_y=True)
   
'''Display results'''

#Show graphs
fig2Filtered.show()
fig.show()

#Show interactive table
print("Top 25 Most Rated Directors by Your Avg Rating")
qgrid.show_grid(dfDirDF)
